<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Multiple Plugins for Decison Making

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets
* Use the API configuration to build the Semantic Kernel orchestrator
* The configuration builder will retrieve the Bing Search API Key

In [1]:
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha"
#r "nuget: Microsoft.Bing.Search.WebSearch, 1.0.0"
#r "nuget: Microsoft.ML, 3.0.1"
#r "nuget: Microsoft.ML.FastTree, 3.0.1"
#r "nuget: Microsoft.Extensions.ML, 3.0.1"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Bing.WebSearch;
using System.ComponentModel;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
var bingSearchAPIKey = config["BingSearch:APIKey"];

Installed Packages Microsoft.Bing.Search.WebSearch, 1.0.0 Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.Extensions.ML, 3.0.1 Microsoft.ML, 3.0.1 Microsoft.ML.FastTree, 3.0.1 Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Plugins.Core, 1.11.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.11.1-alpha

In [2]:
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .Build();

// Import the BaseballHallOfFamePlugin.cs file
#!import ../Classes/BaseballHallOfFamePlugin.cs

// Add the native plugin with the three native functions to the semantic kernel
semanticKernel.ImportPluginFromType<BaseballHallOfFamePlugin>();

### Step 2 - Introduce Sports Decision Scenario 

<img width ="1024x" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SportsDecisionScenario.png">

### Step 3 - Background Information

In [3]:
// Background Information on what is happening & components

// 1) Directory of the historical baseball data statistics
string filePathMLBBaseballBatters = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Data/MLBBaseballBattersPositionPlayers.csv";
// Read the batters data from the CSV file
var batters = File.ReadAllLines(filePathMLBBaseballBatters)
                        .Skip(1)
                        .Select(v => MLBBaseballBatter.FromCsv(v))
                        .ToList();

// 2) A custom decision rules engine crafted by a Machine Learning model
// Machine Learing Model and Prediction Engine                       
string modelPath = Directory.GetParent(Directory.GetCurrentDirectory()) + "/Models/InductedToHoF-GeneralizedAdditiveModels.mlnet";
var mlContext = new MLContext(seed: 100);
ITransformer loadedModel;
DataViewSchema schema;

using (var stream = File.OpenRead(modelPath))
{
    loadedModel = mlContext.Model.Load(stream, out schema);
}
TransformerChain<ITransformer> transfomerChain = (TransformerChain<ITransformer>)loadedModel;

var predEngineInductedToHallOfFame = mlContext.Model.CreatePredictionEngine<MLBBaseballBatter, MLBHOFPrediction>(transfomerChain);

// 3) Predict if Mike Trout will be inducted to the Hall of Fame
var mikeTroutStats = batters.Where(b => b.FullPlayerName == "Mike Trout").FirstOrDefault();
Console.WriteLine(mikeTroutStats.HR);
var prediction = predEngineInductedToHallOfFame.Predict(mikeTroutStats);
Console.WriteLine(prediction.Probability);

350
0.69909585


### Step 4 - Make Hall of Fame Decision only using Web Search Results 

In [4]:
// Use WebSearch to find information on Mike Trout

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Provide sources of information and explain your reasoning.
Ensure to cite all the sources using [number] notation of each URL after the reference in order.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the available data and analysis, I recommend that Mike Trout should indeed be inducted into the Hall of Fame when he becomes eligible. Here is the reasoning behind this recommendation:

1. **Statistical Achievements**: Mike Trout has amassed impressive statistics over his 12-year career, with a batting average of .303, 350 home runs, and 896 RBIs. His slugging percentage stands at .587, and he has accumulated 2,991 total bases [4]. These numbers not only highlight his consistent performance but also his ability to contribute significantly to his team's offensive strength.

2. **Awards and Accolades**: Trout has been an All-Star nine times and has won a total of 21 major awards, including three MVP titles. Such recognitions are indicative of his excellence and impact in the sport [4].

3. **Comparative Analysis**: The probability analysis from the Machine Learning model suggests that Mike Trout has a 69.91% chance of being inducted into the Hall of Fame based on his current sta

### Step 5 - Make Hall of Fame Decision only using Baseball Statistics 

In [5]:
// Use Baseball statistics to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Provide as many baseball statistics as possible.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

### Mike Trout's Baseball Statistics

Here are the key statistics for Mike Trout, which could be considered in the decision-making process for his potential induction into the Hall of Fame:

- **Years Played:** 12
- **At Bats (AB):** 5,094
- **Runs (R):** 1,052
- **Hits (H):** 1,543
- **Doubles:** 296
- **Triples:** 51
- **Home Runs (HR):** 350
- **Runs Batted In (RBI):** 896
- **Stolen Bases (SB):** 204
- **Batting Average:** .303
- **Slugging Percentage:** .587
- **All-Star Appearances:** 9
- **Total Bases (TB):** 2,991
- **Total Player Awards:** 21
- **Last Year Played:** 2022

### Analysis:
Mike Trout has compiled impressive statistics over his 12-year career. His batting average (.303) and slugging percentage (.587) are indicative of a high-performing player. He has hit over 350 home runs and has been selected for the All-Star game nine times, reflecting his consistent excellence and recognition among his peers and baseball analysts.

His total player awards, which stand at 21, fu

### Step 6 - Make Hall of Fame Decision only using a Machine Learning Model

In [6]:
// Use Machine Learning Prediction to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the Hall of Fame? 
Use a machine learning model to get the probability.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the machine learning model, Mike Trout has a 69.91% probability of being inducted into the Hall of Fame. Given this high probability and his impressive statistics, it would be reasonable to recommend that Mike Trout should be considered for Hall of Fame induction.

### Step 7 - Make Hall of Fame Decision using all of the Available Tools (functions)

In [7]:
// Use all of the available tools (functions) to determine if Mike Trout should be inducted to the Hall of Fame

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.6, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0,
    // Enable Auto Invoking of Kernel Functions
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

var decisionPromptTemplateString = """
What decision would you recommend in determining if {{$baseBallPlayer}} should make the hall of fame? 
Use all of the tools available at your disposal to arrive at a comprehensive decision.
You must cite all the sources using [number] notation of each URL after the reference in order.
The sources should appear in order on the bottom of the decision.
""";
var kernelArguments = new KernelArguments(openAIPromptExecutionSettings)
{
    ["baseBallPlayer"] = "Mike Trout"
};

// Now with Auto Invoking of Kernel Functions, let the magic happen
await foreach (var streamChunk in semanticKernel.InvokePromptStreamingAsync(decisionPromptTemplateString, kernelArguments))
{
   Console.Write(streamChunk);
}

Based on the comprehensive analysis, Mike Trout should be considered a strong candidate for the Hall of Fame. His playing statistics alone present a compelling case: over 12 seasons, he has achieved a batting average of 0.303, hit 350 home runs, and garnered 896 RBIs. Trout has also accumulated 204 stolen bases and a slugging percentage of 0.587 [4]. His performance has earned him 9 All-Star appearances and 21 player awards, underscoring his excellence and consistency in the field [1].

Moreover, Trout's eligibility for the Hall of Fame is confirmed as he has played in more than ten Major League championship seasons, fulfilling the necessary criteria for consideration once he retires and undergoes the five-year waiting period [2]. Although injuries have impacted his ability to play full seasons, this does not significantly detract from his overall achievements and his standing as one of the best players in baseball during his era [3].

The machine learning model predicts a 69.91% proba

### Step 8 - Understanding Auto Function Calling Behavior

How does Semantic Kernel and the LLM know to call the function? What if there are multiple functions or plugins, how does LLM signal Semantic Kernel which function to call? 

Executing the cell below will illustrate that the invocation of the functions is not magic, but guided by the descriptions of the plugin function descriptions and parameter descriptions. Note that the Search function is described as "Perform a Web Search". This provides a big clue to Semantic Kernel that this a possible tool (function) to use to extract current information.

In [10]:
// Method for printing the functions metadata
private void PrintFunction(KernelFunctionMetadata func)
{
    Console.WriteLine($"Plugin: {func.PluginName}");
    Console.WriteLine($"  Function Name: {func.Name}");
    Console.WriteLine($"  Function Description: {func.Description}");

    if (func.Parameters.Count > 0)
    {
        Console.WriteLine("    Function Parameters:");
        foreach (var p in func.Parameters)
        {
            Console.WriteLine($"      - {p.Name}: {p.Description}");
            Console.WriteLine($"        default: '{p.DefaultValue}'");
        }
    }

    Console.WriteLine();
}
// Get the functions metadata
var functions = semanticKernel.Plugins.GetFunctionsMetadata();

foreach (KernelFunctionMetadata func in functions)
{
    PrintFunction(func);
}

Plugin: BaseballHallOfFamePlugin
  Function Name: GetPlayerProbabilityOfHallOfFame
  Function Description: Retrieves a Machine Learning probability of a player being Inducted to Hall Of Fame using a the player's baseball statistics.
    Function Parameters:
      - baseballStatistics: The statistics of the baseball player.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetBaseballPlayerStats
  Function Description: Retrieves baseball statistics for a given baseball player.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

Plugin: BaseballHallOfFamePlugin
  Function Name: GetWebSearchResults
  Function Description: Retrieves the web search results with identified URL sources for a given baseball player.
    Function Parameters:
      - nameOfBaseballPlayer: The name of the baseball player to search for.
        default: ''

